In [ ]:
#Q1 MLP with Gaussian likelihood for pneumoniamnist
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

#data1 = np.load('drive/MyDrive/Colab Notebooks/PRNN_A1_DATA/pneumoniamnist.npz')
data1 = np.load('/content/drive/MyDrive/PRNN/pneumoniamnist.npz')
#data2 = np.load('drive/MyDrive/Colab Notebooks/PRNN_A1_DATA/bloodmnist.npz')
print(data1.files)
print(data2.files)

for key, value in data1.items():
  print(key)
  print(data1[key].shape)
for key, value in data2.items():
  print(key)
  print(data2[key].shape)

# Reshape images and save in numpy arrays
train1_data = np.reshape(data1['train_images'],(4708,784))
train1_labels = np.reshape(data1['train_labels'],(4708))

print(train1_data.shape)
print(train1_labels.shape)
test1_data = np.reshape(data1['test_images'],(624,784))
test1_labels = np.reshape(data1['test_labels'],(624))
print(test1_data.shape)
print(test1_labels.shape)

# Gaussian Random Vector with Maximum Likelihood Estimate
# Sample Mean Calculation
mu0 = np.zeros(784)
mu1 = np.zeros(784)
N0 = 0
N1 = 0
for i in range(len(train1_labels)):
  if train1_labels[i] == 0:
    mu0 += train1_data[i]
    N0+=1
  else:
    mu1 += train1_data[i]
    N1+=1 
mu0 = mu0/N0
mu1 = mu1/N1
print(N0)
print(N1)

# Sample Variance Calculation 
sig0 = np.zeros((784,784))
sig1 = np.zeros((784,784))

for i in range(len(train1_labels)):
  if train1_labels[i] == 0:
    sig0 += (1/N0)* np.outer(train1_data[i]-mu0,train1_data[i].T - mu0.T )
  else:  
    sig1 += (1/N1)*np.outer(train1_data[i]-mu1,train1_data[i].T - mu1.T )
#print(np.log(np.linalg.det(sig0/9.5)))
#print(np.log(np.linalg.det(sig1/9.5)))
#print(np.linalg.inv(sig0))
#print(np.outer(train1_data[0]-mu0,train1_data[0].T - mu0.T ).shape)

test1_label_predict_grv = np.zeros(len(test1_labels))
print(test1_label_predict_grv.shape)
inv0 = np.linalg.inv(sig0)
inv1 = np.linalg.inv(sig1)
print(inv0.shape)
print(inv1.shape)
d = np.log(np.linalg.det(sig0/9.5)) - np.log(np.linalg.det(sig1/9.5))
print(d)
print(np.dot(inv1,(test1_data[0]-mu1)).shape)
c = 0
for i in range(len(test1_labels)):
  if np.dot((test1_data[i]-mu0),np.dot(inv0,(test1_data[i]-mu0))) - np.dot((test1_data[i]-mu1),np.dot(inv1,(test1_data[i]-mu1))) + d > 0  :
    test1_label_predict_grv[i] = 1
  else:
    test1_label_predict_grv[i] = 0

test1_label_predict_grv = test1_label_predict_grv.astype(int)
test1_labels = test1_labels.astype(int)

for i,x in enumerate(test1_labels):
  if x == test1_label_predict_grv[i]:
    c+=1
print(c/len(test1_labels)*100)    

def compute_confusion_matrix(true, pred):

  K = len(np.unique(true)) # Number of classes 
  result = np.zeros((K, K))

  for i in range(len(true)):
    result[true[i]][pred[i]] += 1
  tn = result[0][0] 
  fn = result[0][1] 
  fp = result[1][0] 
  tp = result[1][1]
  acc = (tp+tn)/(tp+tn+fp+fn)
  f1 = tp/(tp+0.5*(fp+fn)) 
  print("Accuracy = ", acc)
  print("F-1 Score = ", f1)
  return result

print(compute_confusion_matrix(test1_labels,test1_label_predict_grv))

In [ ]:
#MLE with Naive Bayes Gaussian likehood for pneumoniamnist
sig0nb = np.zeros(784)
sig1nb = np.zeros(784)
for i in range(len(train1_labels)):
  if train1_labels[i] == 0:
    sig0nb+= (1/N0)*(mu0 - train1_data[i])**2
  else:
    sig1nb+= (1/N1)*(mu1 - train1_data[i])**2 

test1_labels_predict_nb = np.zeros(784)
sig0_inv = 1/sig0nb
sig1_inv = 1/sig1nb
for i in range(len(test1_labels)):
  if np.dot((test1_data[i] - mu0),(sig0_inv *(test1_data[i] - mu0))) > np.dot((test1_data[i] - mu1),(sig1_inv *(test1_data[i] - mu1))):
    test1_labels_predict_nb[i] = 1
c= 0 

test1_labels_predict_nb = test1_labels_predict_nb.astype(int)
test1_labels = test1_labels.astype(int)

for i,x in enumerate(test1_labels):
  if x == test1_labels_predict_nb[i]:
    c+=1
print(c/len(test1_labels)*100)


def compute_confusion_matrix(true, pred):

  K = len(np.unique(true)) # Number of classes 
  result = np.zeros((K, K))

  for i in range(len(true)):
    result[true[i]][pred[i]] += 1
  tn = result[0][0] 
  fn = result[0][1] 
  fp = result[1][0] 
  tp = result[1][1]
  acc = (tp+tn)/(tp+tn+fp+fn)
  f1 = tp/(tp+0.5*(fp+fn)) 
  print("Accuracy = ", acc)
  print("F-1 Score = ", f1)
  return result

print(compute_confusion_matrix(test1_labels,test1_labels_predict_nb))


In [1]:
#MLE with Gaussian Random Vector likelihood for bloodmnist
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data2 = np.load('../input/bloodmnist/bloodmnist.npz')
print(data2.files)

for key, value in data2.items():
  print(key)
  print(data2[key].shape)

import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray


def process_data(path):
    dataset = np.load(path)
    train_x, train_y = np.reshape(
        rgb2gray(dataset['train_images']), (11959, 784, 1)), dataset['train_labels']
    val_x, val_y = np.reshape(rgb2gray(dataset['val_images']),
                              (1712, 784, 1)), dataset['val_labels']
    test_x, test_y = np.reshape(
        rgb2gray(dataset['test_images']), (3421, 784, 1)), dataset['test_labels']

    return train_x, train_y, test_x, test_y


def estimate_class_conditionals(train_x, train_y):
    n = train_x.shape[1]
    mu0 = np.zeros((n, 1))
    mu1 = np.zeros((n, 1))
    mu2 = np.zeros((n, 1))
    mu3 = np.zeros((n, 1))
    mu4 = np.zeros((n, 1))
    mu5 = np.zeros((n, 1))
    mu6 = np.zeros((n, 1))
    mu7 = np.zeros((n, 1))
    sig0 = np.zeros((n, n), dtype=np.double)
    sig1 = np.zeros((n, n), dtype=np.double)
    sig2 = np.zeros((n, n), dtype=np.double)
    sig3 = np.zeros((n, n), dtype=np.double)
    sig4 = np.zeros((n, n), dtype=np.double)
    sig5 = np.zeros((n, n), dtype=np.double)
    sig6 = np.zeros((n, n), dtype=np.double)
    sig7 = np.zeros((n, n), dtype=np.double)
    n0, n1, n2, n3, n4, n5, n6, n7 = 0, 0, 0, 0, 0, 0, 0, 0
    n = train_x.shape[0]

    print("[INFO] Estimating class conditional densities")

    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            mu0 += train_x[i]
            n0 += 1
        elif(train_y[i] == 1):
            mu1 += train_x[i]
            n1 += 1
        elif(train_y[i] == 2):
            mu2 += train_x[i]
            n2 += 1
        if(train_y[i] == 3):
            mu3 += train_x[i]
            n3 += 1
        elif(train_y[i] == 4):
            mu4 += train_x[i]
            n4 += 1
        elif(train_y[i] == 5):
            mu5 += train_x[i]
            n5 += 1
        if(train_y[i] == 6):
            mu6 += train_x[i]
            n6 += 1
        else:
            mu7 += train_x[i]
            n7 += 1

    mu0 *= 1.0/n0
    mu1 *= 1.0/n1
    mu2 *= 1.0/n2
    mu3 *= 1.0/n3
    mu4 *= 1.0/n4
    mu5 *= 1.0/n5
    mu6 *= 1.0/n6
    mu7 *= 1.0/n7
    means = [mu0, mu1, mu2, mu3, mu4, mu5, mu6, mu7]
    print(n0,n1,n2,n3,n4,n5,n6,n7)
    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            sig0 += np.outer(train_x[i]-mu0, (train_x[i]-mu0).T)
        elif(train_y[i] == 1):
            sig1 += np.outer(train_x[i]-mu1, (train_x[i]-mu1).T)
        elif(train_y[i] == 2):
            sig2 += np.outer(train_x[i]-mu2, (train_x[i]-mu2).T)
        elif(train_y[i] == 3):
            sig3 += np.outer(train_x[i]-mu3, (train_x[i]-mu3).T)
        elif(train_y[i] == 4):
            sig4 += np.outer(train_x[i]-mu4, (train_x[i]-mu4).T)
        elif(train_y[i] == 5):
            sig5 += np.outer(train_x[i]-mu5, (train_x[i]-mu5).T)
        elif(train_y[i] == 6):
            sig6 += np.outer(train_x[i]-mu6, (train_x[i]-mu6).T)
        else:
            sig7 += np.outer(train_x[i]-mu7, (train_x[i]-mu7).T)

    sig0 *= 1.0/n0
    sig1 *= 1.0/n1
    sig2 *= 1.0/n2
    sig3 *= 1.0/n3
    sig4 *= 1.0/n4
    sig5 *= 1.0/n5
    sig6 *= 1.0/n6
    sig7 *= 1.0/n7
    covariances = [sig0, sig1, sig2, sig3, sig4, sig5, sig6, sig7]

    p0 = n0/n
    p1 = n1/n
    p2 = n2/n
    p3 = n3/n
    p4 = n4/n
    p5 = n5/n
    p6 = n6/n
    p7 = n7/n
    priors = [p0, p1, p2, p3, p4, p5, p6, p7]

    (sign0, logdet0) = np.linalg.slogdet(sig0)
    (sign1, logdet1) = np.linalg.slogdet(sig1)
    (sign2, logdet2) = np.linalg.slogdet(sig2)
    (sign3, logdet3) = np.linalg.slogdet(sig3)
    (sign4, logdet4) = np.linalg.slogdet(sig4)
    (sign5, logdet5) = np.linalg.slogdet(sig5)
    (sign6, logdet6) = np.linalg.slogdet(sig6)
    (sign7, logdet7) = np.linalg.slogdet(sig7)
    
    sigma0_inv = np.linalg.pinv(sig0)
    sigma1_inv = np.linalg.pinv(sig1)
    sigma2_inv = np.linalg.pinv(sig2)
    sigma3_inv = np.linalg.pinv(sig3)
    sigma4_inv = np.linalg.pinv(sig4)
    sigma5_inv = np.linalg.pinv(sig5)
    sigma6_inv = np.linalg.pinv(sig6)
    sigma7_inv = np.linalg.pinv(sig7)
    sigma_inv = [sigma0_inv,sigma1_inv,sigma2_inv,sigma3_inv,sigma4_inv,sigma5_inv,sigma6_inv,sigma7_inv]
    logdets = [logdet0, logdet1, logdet2, logdet3, logdet4, logdet5, logdet6, logdet7]

    return means, covariances, logdets, priors, sigma_inv


def calc_log_posterior(X, mu, sigma, prior, logdet,sigma_inv):
    return np.log(prior) - 0.5*logdet - 0.5*np.dot((X-mu).T,
                                                      np.dot(sigma_inv, X-mu))


def confusion_matrix(actual, predicted):

    # extract the different classes
    classes = np.unique(actual)
    # initialize the confusion matrix
    confmat = np.zeros((len(classes), len(classes)))

    # loop across the different combinations of actual / predicted classes
    for i in range(len(classes)):
        for j in range(len(classes)):
            # count the number of instances in each combination of actual / predicted classes
            confmat[i, j] = np.sum((actual == classes[i]) & (predicted == classes[j]))

    return confmat


def testing_bayes(test_x, test_y, means, covariances, logdets, priors,sigma_inv):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    K = len(priors)  # Number of classes
    print(K)
    print("[INFO] Calculating accuracy and F1 score on test data")
    #print(means,logdets)
    predicted_labels = np.zeros((test_y.shape[0], 1))
    print(test_y.shape[0])
    for i in range(test_y.shape[0]):
        vals = []
        for k in range(K):
            vals.append(calc_log_posterior(test_x[i, :, :], means[k], covariances[k], priors[k], logdets[k],sigma_inv[k]))
        predicted_labels[i] = np.argmax(vals)
    conf_mat = confusion_matrix(test_y, predicted_labels)
    print(conf_mat)

    acc = np.sum(np.diag(conf_mat))/np.sum(conf_mat)
    #f1 = tp/(tp+0.5*(fp+fn))
    print("Accuracy = ", acc)
    #print("F-1 Score = ", f1)

    return


def main():
    #path = '/content/drive/MyDrive/PRNN/bloodmnist.npz'
    path = "../input/bloodmnist/bloodmnist.npz" #path for rahul
    train_x, train_y, test_x, test_y = process_data(path)
    means, covariances, logdets, priors,sigma_inv = estimate_class_conditionals(train_x, train_y)
    print("Ok")
    testing_bayes(test_x, test_y, means, covariances, logdets, priors,sigma_inv)


if __name__ == "__main__":
    main()

In [ ]:
#Naive Bayes Gaussian for bloodmnist

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data2 = np.load('../input/bloodmnist/bloodmnist.npz')
print(data2.files)

for key, value in data2.items():
  print(key)
  print(data2[key].shape)

import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray


def process_data(path):
    dataset = np.load(path)
    train_x, train_y = np.reshape(
        rgb2gray(dataset['train_images']), (11959, 784, 1)), dataset['train_labels']
    val_x, val_y = np.reshape(rgb2gray(dataset['val_images']),
                              (1712, 784, 1)), dataset['val_labels']
    test_x, test_y = np.reshape(
        rgb2gray(dataset['test_images']), (3421, 784, 1)), dataset['test_labels']

    return train_x, train_y, test_x, test_y


def estimate_class_conditionals(train_x, train_y):
    n = train_x.shape[1]
    mu0 = np.zeros((n, 1))
    mu1 = np.zeros((n, 1))
    mu2 = np.zeros((n, 1))
    mu3 = np.zeros((n, 1))
    mu4 = np.zeros((n, 1))
    mu5 = np.zeros((n, 1))
    mu6 = np.zeros((n, 1))
    mu7 = np.zeros((n, 1))
    sig0 = np.zeros((n, n), dtype=np.double)
    sig1 = np.zeros((n, n), dtype=np.double)
    sig2 = np.zeros((n, n), dtype=np.double)
    sig3 = np.zeros((n, n), dtype=np.double)
    sig4 = np.zeros((n, n), dtype=np.double)
    sig5 = np.zeros((n, n), dtype=np.double)
    sig6 = np.zeros((n, n), dtype=np.double)
    sig7 = np.zeros((n, n), dtype=np.double)
    n0, n1, n2, n3, n4, n5, n6, n7 = 0, 0, 0, 0, 0, 0, 0, 0
    n = train_x.shape[0]

    print("[INFO] Estimating class conditional densities")

    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            mu0 += train_x[i]
            n0 += 1
        elif(train_y[i] == 1):
            mu1 += train_x[i]
            n1 += 1
        elif(train_y[i] == 2):
            mu2 += train_x[i]
            n2 += 1
        if(train_y[i] == 3):
            mu3 += train_x[i]
            n3 += 1
        elif(train_y[i] == 4):
            mu4 += train_x[i]
            n4 += 1
        elif(train_y[i] == 5):
            mu5 += train_x[i]
            n5 += 1
        if(train_y[i] == 6):
            mu6 += train_x[i]
            n6 += 1
        else:
            mu7 += train_x[i]
            n7 += 1

    mu0 *= 1.0/n0
    mu1 *= 1.0/n1
    mu2 *= 1.0/n2
    mu3 *= 1.0/n3
    mu4 *= 1.0/n4
    mu5 *= 1.0/n5
    mu6 *= 1.0/n6
    mu7 *= 1.0/n7
    means = [mu0, mu1, mu2, mu3, mu4, mu5, mu6, mu7]
    print(n0,n1,n2,n3,n4,n5,n6,n7)
    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            sig0 += np.outer(train_x[i]-mu0, (train_x[i]-mu0).T)
        elif(train_y[i] == 1):
            sig1 += np.outer(train_x[i]-mu1, (train_x[i]-mu1).T)
        elif(train_y[i] == 2):
            sig2 += np.outer(train_x[i]-mu2, (train_x[i]-mu2).T)
        elif(train_y[i] == 3):
            sig3 += np.outer(train_x[i]-mu3, (train_x[i]-mu3).T)
        elif(train_y[i] == 4):
            sig4 += np.outer(train_x[i]-mu4, (train_x[i]-mu4).T)
        elif(train_y[i] == 5):
            sig5 += np.outer(train_x[i]-mu5, (train_x[i]-mu5).T)
        elif(train_y[i] == 6):
            sig6 += np.outer(train_x[i]-mu6, (train_x[i]-mu6).T)
        else:
            sig7 += np.outer(train_x[i]-mu7, (train_x[i]-mu7).T)

    sig0 *= 1.0/n0
    sig1 *= 1.0/n1
    sig2 *= 1.0/n2
    sig3 *= 1.0/n3
    sig4 *= 1.0/n4
    sig5 *= 1.0/n5
    sig6 *= 1.0/n6
    sig7 *= 1.0/n7
    covariances = [sig0, sig1, sig2, sig3, sig4, sig5, sig6, sig7]

    p0 = n0/n
    p1 = n1/n
    p2 = n2/n
    p3 = n3/n
    p4 = n4/n
    p5 = n5/n
    p6 = n6/n
    p7 = n7/n
    priors = [p0, p1, p2, p3, p4, p5, p6, p7]

    (sign0, logdet0) = np.linalg.slogdet(sig0)
    (sign1, logdet1) = np.linalg.slogdet(sig1)
    (sign2, logdet2) = np.linalg.slogdet(sig2)
    (sign3, logdet3) = np.linalg.slogdet(sig3)
    (sign4, logdet4) = np.linalg.slogdet(sig4)
    (sign5, logdet5) = np.linalg.slogdet(sig5)
    (sign6, logdet6) = np.linalg.slogdet(sig6)
    (sign7, logdet7) = np.linalg.slogdet(sig7)
    
    sigma0_inv = np.linalg.pinv(sig0)
    sigma1_inv = np.linalg.pinv(sig1)
    sigma2_inv = np.linalg.pinv(sig2)
    sigma3_inv = np.linalg.pinv(sig3)
    sigma4_inv = np.linalg.pinv(sig4)
    sigma5_inv = np.linalg.pinv(sig5)
    sigma6_inv = np.linalg.pinv(sig6)
    sigma7_inv = np.linalg.pinv(sig7)
    sigma_inv = [sigma0_inv,sigma1_inv,sigma2_inv,sigma3_inv,sigma4_inv,sigma5_inv,sigma6_inv,sigma7_inv]
    logdets = [logdet0, logdet1, logdet2, logdet3, logdet4, logdet5, logdet6, logdet7]

    return means, covariances, logdets, priors, sigma_inv


def calc_log_posterior(X, mu, sigma, prior, logdet,sigma_inv):
    return np.log(prior) - 0.5*logdet - 0.5*np.dot((X-mu).T,
                                                      np.dot(sigma_inv, X-mu))


def confusion_matrix(actual, predicted):

    # extract the different classes
    classes = np.unique(actual)
    # initialize the confusion matrix
    confmat = np.zeros((len(classes), len(classes)))

    # loop across the different combinations of actual / predicted classes
    for i in range(len(classes)):
        for j in range(len(classes)):
            # count the number of instances in each combination of actual / predicted classes
            confmat[i, j] = np.sum((actual == classes[i]) & (predicted == classes[j]))

    return confmat


def testing_bayes(test_x, test_y, means, covariances, logdets, priors,sigma_inv):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    K = len(priors)  # Number of classes
    print(K)
    print("[INFO] Calculating accuracy and F1 score on test data")
    #print(means,logdets)
    predicted_labels = np.zeros((test_y.shape[0], 1))
    print(test_y.shape[0])
    for i in range(test_y.shape[0]):
        vals = []
        for k in range(K):
            vals.append(calc_log_posterior(test_x[i, :, :], means[k], covariances[k], priors[k], logdets[k],sigma_inv[k]))
        predicted_labels[i] = np.argmax(vals)
    conf_mat = confusion_matrix(test_y, predicted_labels)
    print(conf_mat)

    acc = np.sum(np.diag(conf_mat))/np.sum(conf_mat)
    #f1 = tp/(tp+0.5*(fp+fn))
    print("Accuracy = ", acc)
    #print("F-1 Score = ", f1)

    return


def main():
    #path = '/content/drive/MyDrive/PRNN/bloodmnist.npz'
    path = "../input/bloodmnist/bloodmnist.npz" #path for rahul
    train_x, train_y, test_x, test_y = process_data(path)
    means, covariances, logdets, priors,sigma_inv = estimate_class_conditionals(train_x, train_y)
    print("Ok")
    testing_bayes(test_x, test_y, means, covariances, logdets, priors,sigma_inv)


if __name__ == "__main__":
    main()

In [2]:
#Linear Classification Models on pneumoniamnist with L1,L2, Elastic Regularizers
#Linear Classification With L2 Regularizer

import numpy as np
from numpy import load
import pandas as pd
from skimage.color import rgb2gray
from matplotlib import pyplot
import sklearn.metrics


def initialize_with_zeros(dim):
    w = np.zeros(shape=(dim, 1))
    b = 0
    return w, b

def normalize(X):

    X[:, 1:] = (X[:, 1:] - np.mean(X[:, 1:], axis=0)) / np.std(X[:, 1:], axis=0)

    return X

def propagate(w, b, X, Y):
    # Find the number of training data
    m = X.shape[1]


    # Calculate the predicted output
    A_reg = (np.dot(w.T, X) + b)
    A=np.copy(A_reg)
    A[A_reg>=0]=1
    A[A_reg<0]=-1


    # Calculate the cost function
    cost = (np.sum(((A-Y).T@(A-Y)))+ 0.1 * np.sum(np.abs(w)))  / (2* m)

    # Calculate the gradients
    dw = 1 / m * (np.dot(X, (A - Y).T)+0.1*np.sign(w))
    db = 1 / m * (np.sum(A - Y)+0.1*np.sign(b))

    grads = {"dw": dw,
             "db": db}
    return grads, cost


def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []

    # propagate function will run for a number of iterations
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]

        # Updating w and b by deducting the dw
        # and db times learning rate from the previous
        # w and b

        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the cost function value for each 100 iterations
        if i % 100 == 0:
            costs.append(cost)

    # The final updated parameters
    params = {"w": w,
              "b": b}

    # The final updated gradients
    grads = {"dw": dw,
             "db": db}

    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]

    # Initializing an aray of zeros which has a size of the input
    # These zeros will be replaced by the predicted output
    Y_prediction = np.zeros((1, m))

    # w = w.reshape(X.shape[0], 1)

    # Calculating the predicted output using the Formula 1
    # This will return the values from 0 to 1
    A = (np.dot(w.T, X) + b)

    # Iterating through A and predict an 1 if the value of A
    # is greater than 0.5 and zero otherwise

    Y_prediction[A<0]=-1
    Y_prediction[A>0] =1
    return Y_prediction


def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate):

    X_train=normalize(X_train)
    X_test=normalize(X_test)

    # Initializing the w and b as zeros
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost=True)
    w = parameters["w"]
    b = parameters["b"]

    # Predicting the output for both test and training set
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("Linear Classification With L2 Regularizer")
    # Calculating the training and test set accuracy by comparing
    # the predicted output and the original output
    num_correct = np.sum(Y_prediction_train == Y_train)
    train_accuracy = (num_correct / X_train.shape[1]) * 100
    print("Train accuracy :  {} %".format(train_accuracy))
    num_correct = np.sum(Y_prediction_test == Y_test)
    test_accuracy = (num_correct / X_test.shape[1]) * 100
    print("Test accuracy :  {} %".format(test_accuracy))
    
    F1= sklearn.metrics.f1_score(Y_test[0, :], Y_prediction_test[0, :])
    AUC = sklearn.metrics.roc_auc_score(Y_test[0, :], Y_prediction_test[0, :])

    print('F1 score : {}'.format(F1))
    print('AUC : {}'.format(AUC))
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d


def main():
    data = load('../input/d/datasets/pogurahulraju/pneumoniamnist/pneumoniamnist.npz')

    
    flatten_data = {}
    n1, l, b = data['train_images'].shape
    n2, l, b = data['test_images'].shape
    flatten_data['train_images'] = data['train_images'].reshape(n1, l * b)
    flatten_data['test_images'] = data['test_images'].reshape(n2, l * b)
    #print(flatten_data['train_images'].shape)
    #print(flatten_data['test_images'].shape)

    l_train = data['train_labels']
    l_test = data['test_labels']


    X_train = pd.DataFrame(flatten_data['train_images'])
    X_test = pd.DataFrame(flatten_data['test_images'])
    y_train = pd.DataFrame(l_train)
    y_test = pd.DataFrame(l_test)


    X_train = np.array(X_train.T)/255
    y_train = np.array(y_train.T)
    X_test = np.array(X_test.T)/255
    y_test = np.array(y_test.T)

    y_train = y_train.astype("int")
    y_test = y_test.astype("int")

    y_train[y_train==0]=(0-1)
    y_test[y_test==0]=(0-1)


    d = model(X_train, y_train, X_test, y_test, num_iterations=50, learning_rate=0.2)

    return



if __name__ == '__main__':
    main()

#Linear Classification With L1 Regularizer
import numpy as np
from numpy import load
import pandas as pd
from skimage.color import rgb2gray
from matplotlib import pyplot


def initialize_with_zeros(dim):
    w = np.zeros(shape=(dim, 1))
    b = 0
    return w, b

def normalize(X):

    X[:, 1:] = (X[:, 1:] - np.mean(X[:, 1:], axis=0)) / np.std(X[:, 1:], axis=0)

    return X

def propagate(w, b, X, Y):
    # Find the number of training data
    m = X.shape[1]


    # Calculate the predicted output
    A_reg = (np.dot(w.T, X) + b)
    A=np.copy(A_reg)
    A[A_reg>=0]=1
    A[A_reg<0]=-1


    # Calculate the cost function
    cost = (np.sum(((A-Y).T@(A-Y))) +  0.1 * (np.sum(np.dot(w.T,w))+b**2))/ (2* m)

    # Calculate the gradients
    dw = 1 / m * (np.dot(X, (A - Y).T)+0.1*w)
    db = 1 / m * (np.sum(A - Y)+0.1*b)

    grads = {"dw": dw,
             "db": db}
    return grads, cost


def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []

    # propagate function will run for a number of iterations
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]

        # Updating w and b by deducting the dw
        # and db times learning rate from the previous
        # w and b

        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the cost function value for each 100 iterations
        if i % 100 == 0:
            costs.append(cost)

    # The final updated parameters
    params = {"w": w,
              "b": b}

    # The final updated gradients
    grads = {"dw": dw,
             "db": db}

    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]

    # Initializing an aray of zeros which has a size of the input
    # These zeros will be replaced by the predicted output
    Y_prediction = np.zeros((1, m))

    # w = w.reshape(X.shape[0], 1)

    # Calculating the predicted output using the Formula 1
    # This will return the values from 0 to 1
    A = (np.dot(w.T, X) + b)

    # Iterating through A and predict an 1 if the value of A
    # is greater than 0.5 and zero otherwise

    Y_prediction[A<0]=-1
    Y_prediction[A>0] =1
    return Y_prediction


def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate):

    X_train=normalize(X_train)
    X_test=normalize(X_test)

    # Initializing the w and b as zeros
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost=True)
    w = parameters["w"]
    b = parameters["b"]

    # Predicting the output for both test and training set
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("Linear Classification With L1 Regularizer")    
    # Calculating the training and test set accuracy by comparing
    # the predicted output and the original output
    num_correct = np.sum(Y_prediction_train == Y_train)
    train_accuracy = (num_correct / X_train.shape[1]) * 100
    print("Train accuracy :  {} %".format(train_accuracy))
    num_correct = np.sum(Y_prediction_test == Y_test)
    test_accuracy = (num_correct / X_test.shape[1]) * 100
    print("Test accuracy :  {} %".format(test_accuracy))
    
    F1= sklearn.metrics.f1_score(Y_test[0, :], Y_prediction_test[0, :])
    AUC = sklearn.metrics.roc_auc_score(Y_test[0, :], Y_prediction_test[0, :])
    
    print('F1 score : {}'.format(F1))
    print('AUC : {}'.format(AUC))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d


def main():
    data = load('../input/d/datasets/pogurahulraju/pneumoniamnist/pneumoniamnist.npz')

    flatten_data = {}
    n1, l, b = data['train_images'].shape
    n2, l, b = data['test_images'].shape
    flatten_data['train_images'] = data['train_images'].reshape(n1, l * b)
    flatten_data['test_images'] = data['test_images'].reshape(n2, l * b)
   

    l_train = data['train_labels']
    l_test = data['test_labels']


    X_train = pd.DataFrame(flatten_data['train_images'])
    X_test = pd.DataFrame(flatten_data['test_images'])
    y_train = pd.DataFrame(l_train)
    y_test = pd.DataFrame(l_test)


    X_train = np.array(X_train.T)/255
    y_train = np.array(y_train.T)
    X_test = np.array(X_test.T)/255
    y_test = np.array(y_test.T)

    y_train = y_train.astype("int")
    y_test = y_test.astype("int")

    y_train[y_train==0]=(0-1)
    y_test[y_test==0]=(0-1)


    d = model(X_train, y_train, X_test, y_test, num_iterations=50, learning_rate=0.2)

    return



if __name__ == '__main__':
    main()

#Linear Regression With Elastic Regularizer

import numpy as np
from numpy import load
import pandas as pd
from skimage.color import rgb2gray
from matplotlib import pyplot


def initialize_with_zeros(dim):
    w = np.zeros(shape=(dim, 1))
    b = 0
    return w, b

def normalize(X):

    X[:, 1:] = (X[:, 1:] - np.mean(X[:, 1:], axis=0)) / np.std(X[:, 1:], axis=0)

    return X

def propagate(w, b, X, Y):
    # Find the number of training data
    m = X.shape[1]


    # Calculate the predicted output
    A_reg = (np.dot(w.T, X) + b)
    A=np.copy(A_reg)
    A[A_reg>=0]=1
    A[A_reg<0]=-1


    # Calculate the cost function
    cost = (np.sum(((A-Y).T@(A-Y))) + 0.1 * (np.sum(np.dot(w.T,w))+b**2) + 0.1 * np.sum(np.abs(w))) / (2* m)

    # Calculate the gradients
    dw = 1 / m * (np.dot(X, (A - Y).T)+0.1*w+0.1*np.sign(w))
    db = 1 / m * (np.sum(A - Y)+0.1*b+0.1*np.sign(b))

    grads = {"dw": dw,
             "db": db}
    return grads, cost


def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []

    # propagate function will run for a number of iterations
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]

        # Updating w and b by deducting the dw
        # and db times learning rate from the previous
        # w and b

        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the cost function value for each 100 iterations
        if i % 100 == 0:
            costs.append(cost)

    # The final updated parameters
    params = {"w": w,
              "b": b}

    # The final updated gradients
    grads = {"dw": dw,
             "db": db}

    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]

    # Initializing an aray of zeros which has a size of the input
    # These zeros will be replaced by the predicted output
    Y_prediction = np.zeros((1, m))

    # w = w.reshape(X.shape[0], 1)

    # Calculating the predicted output using the Formula 1
    # This will return the values from 0 to 1
    A = (np.dot(w.T, X) + b)

    # Iterating through A and predict an 1 if the value of A
    # is greater than 0.5 and zero otherwise

    Y_prediction[A<0]=-1
    Y_prediction[A>0] =1
    return Y_prediction


def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate):

    X_train=normalize(X_train)
    X_test=normalize(X_test)

    # Initializing the w and b as zeros
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost=True)
    w = parameters["w"]
    b = parameters["b"]

    # Predicting the output for both test and training set
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    print("Linear Classification With Elastic Regularizer")
    
    # Calculating the training and test set accuracy by comparing
    # the predicted output and the original output
    num_correct = np.sum(Y_prediction_train == Y_train)
    train_accuracy = (num_correct / X_train.shape[1]) * 100
    print("Train accuracy :  {} %".format(train_accuracy))
    num_correct = np.sum(Y_prediction_test == Y_test)
    test_accuracy = (num_correct / X_test.shape[1]) * 100
    print("Test accuracy :  {} %".format(test_accuracy))
    
    F1= sklearn.metrics.f1_score(Y_test[0, :], Y_prediction_test[0, :])
    AUC = sklearn.metrics.roc_auc_score(Y_test[0, :], Y_prediction_test[0, :])

    print('F1 score : {}'.format(F1))
    print('AUC : {}'.format(AUC))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d


def main():
    data = load('../input/d/datasets/pogurahulraju/pneumoniamnist/pneumoniamnist.npz')
    
    flatten_data = {}
    n1, l, b = data['train_images'].shape
    n2, l, b = data['test_images'].shape
    flatten_data['train_images'] = data['train_images'].reshape(n1, l * b)
    flatten_data['test_images'] = data['test_images'].reshape(n2, l * b)

    l_train = data['train_labels']
    l_test = data['test_labels']


    X_train = pd.DataFrame(flatten_data['train_images'])
    X_test = pd.DataFrame(flatten_data['test_images'])
    y_train = pd.DataFrame(l_train)
    y_test = pd.DataFrame(l_test)


    X_train = np.array(X_train.T)/255
    y_train = np.array(y_train.T)
    X_test = np.array(X_test.T)/255
    y_test = np.array(y_test.T)

    y_train = y_train.astype("int")
    y_test = y_test.astype("int")

    y_train[y_train==0]=(0-1)
    y_test[y_test==0]=(0-1)


    d = model(X_train, y_train, X_test, y_test, num_iterations=50, learning_rate=0.2)

    return



if __name__ == '__main__':
    main()

In [ ]:
#K Nearest Neighbours, Parzen window estimation on pneumoniamnist
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

"""
Evaluation Metrics for Classification

Confusion Matrix, Accuracy, F1 Score and AUC
"""

# Confusion Matrix
def confMatrix(test_y, test_r, classes) :
    conf_mat = pd.DataFrame(np.zeros((classes.shape[0], classes.shape[0])), classes, classes)
    for i in range(test_y.shape[0]):
        conf_mat[test_r[i]][test_y[i]] += 1
    return conf_mat

# Accuracy
def accuracy(conf_mat) :
    return np.trace(conf_mat)/conf_mat.values.sum()

# F1 Score
def f1Score(conf_mat) :
    return pd.DataFrame(2*np.diag(conf_mat), conf_mat.index, ["f1-score"]).div(pd.DataFrame(conf_mat.sum(axis = 0) + conf_mat.sum(axis = 1), columns=["f1-score"]))

"""
Distance Functions

Returns the distance between the two objects
"""
# Euclidean Distance Function
def eucDist(a, b):
    return np.linalg.norm(a - b)
    
# Manhattan Distance Function
def manDist(a, b):
    return np.sum(np.abs(a - b))

"""
Nearest Neighbours Classifiers

Returns the probabilities of the labels of the test dataset from the nearest neighbour classfication
using the training data
"""

# Nearest Neighbour Classification
def NNClassifier(trainX, trainY, testX, distFunc = eucDist):
    minDist = np.inf
    argMin = -1
    testR = np.zeros(testX.shape[0])
    for i in range(testX.shape[0]):
        for j in range(trainX.shape[0]):
            Dist = distFunc(trainX[j], testX[i])
            if minDist > Dist:
                minDist = Dist
                argMin = j
        testR[i] = trainY[argMin]
    return testR

# K Nearest Neighbour Classification
def kNNClassifier(trainX, trainY, testX, nClasses, k = 3, distFunc = eucDist):
    dist = np.zeros((trainX.shape[0],2))
    probR = np.zeros((testX.shape[0], nClasses))
    for i in range(testX.shape[0]):
        for j in range(trainX.shape[0]):
            dist[j][0] = distFunc(testX[i], trainX[j])
            dist[j][1] = j
        dist = dist[np.argsort(dist[:, 0])]
        for j in range(k):
            probR[i][trainY[int(dist[j][1])]] += 1/k
    return probR

"""
Window Functions

Returns the weight of the given object within the window function
"""
# HyperCube Window Function
def HCWindow(x, a, h):
    for i in range(a.shape[0]):
        if abs(a[i] - x[i]) > h/2:
            return 0
    return 1

# Gaussian Window Function
def GWindow(x, a, h):
    return np.exp(-eucDist(x,a)/(2*h*h))

"""
Parzen Window Classifiers

Returns the probabilities of the labels for the test dataset from the Parzen Window Classifier
using the training Data
"""
def PWClassifier(trainX, trainY, testX, nClasses, h = 0.3, windFunc = GWindow):
    cClass = np.zeros(nClasses)
    for j in trainY:
        cClass[j] += 1
    
    probDensR = np.zeros((testX.shape[0], nClasses))
    for i in range(testX.shape[0]):
        for j in range(trainX.shape[0]):
            probDensR[i][trainY[j]] += windFunc(testX[i], trainX[j], h)/cClass[trainY[j]]
    
    return probDensR

pnuData = np.load("../input/classification-dataset/pneumoniamnist.npz")
bldData = np.load("../input/classification-dataset/bloodmnist.npz")

pnuTrShp = pnuData['train_images'].shape
pnuTrainX = pnuData['train_images'].reshape(pnuTrShp[0], pnuTrShp[1]*pnuTrShp[2])
pnuTrainY = pnuData['train_labels']

pnuTeShp = pnuData['test_images'].shape
pnuTestX = pnuData['test_images'].reshape(pnuTeShp[0], pnuTeShp[1]*pnuTeShp[2])
pnuTestY = pnuData['test_labels']

pnuTrainX = pnuTrainX/255
pnuTestX = pnuTestX/255

pnuClasses = np.union1d(pnuTrainY, pnuTestY)

pnukNNProbR = kNNClassifier(pnuTrainX, pnuTrainY, pnuTestX, len(pnuClasses))
pnukNNTestR = pnukNNProbR.argmax(axis = 1)

pnukNNCM = confMatrix(pnuTestY, pnukNNTestR, pnuClasses)
print("pnu kNN Accuracy:", accuracy(pnukNNCM))
print("pnu kNN F1 Score:", f1Score(pnukNNCM))
print("pnu kNN AUC Score:", roc_auc_score(pnukNNTestR, pnuTestY))

"""Accuracy is around 83 percent"""

pnuPWProbR = PWClassifier(pnuTrainX, pnuTrainY, pnuTestX, len(pnuClasses))
pnuPWTestR = pnuPWProbR.argmax(axis = 1)

pnuPWCM = confMatrix(pnuTestY, pnuPWTestR, pnuClasses)
print("pnu PW Accuracy:", accuracy(pnuPWCM))
print("pnu PW F1 Score:", f1Score(pnuPWCM))
print("pnu PW AUC Score:", roc_auc_score(pnuPWTestR, pnuTestY))

"""Accuracy is around 47 percent"""

rgb2gray = [0.299, 0.587, 0.114]

bldTrShp = bldData['train_images'].shape
bldTrXC = bldData['train_images'].reshape(bldTrShp[0], bldTrShp[1]*bldTrShp[2]*bldTrShp[3])
bldTrXG = np.average(bldData['train_images'], axis = 3, weights = rgb2gray).reshape(bldTrShp[0], bldTrShp[1]*bldTrShp[2])
bldTrY = bldData['train_labels']

bldTeShp = bldData['test_images'].shape
bldTeXC = bldData['test_images'].reshape(bldTeShp[0], bldTeShp[1]*bldTeShp[2]*bldTeShp[3])
bldTeXG = np.average(bldData['test_images'], axis = 3, weights = rgb2gray).reshape(bldTeShp[0], bldTeShp[1]*bldTeShp[2])
bldTeY = bldData['test_labels']

bldTrXC = bldTrXC/255
bldTrXG = bldTrXG/255

bldTeXC = bldTeXC/255
bldTeXG = bldTeXG/255

bldClasses = np.union1d(bldTrY, bldTeY)

bldkNNProbR = kNNClassifier(bldTrXG, bldTrY, bldTeXG, len(bldClasses))
bldkNNTestR = bldkNNProbR.argmax(axis = 1)

bldkNNCM = confMatrix(bldTeY, bldkNNTestR, bldClasses)
print("bld kNN Accuracy:", accuracy(bldkNNCM))
print("bld kNN F1 Score:", f1Score(bldkNNCM))
print("bld kNN AUC Score:", roc_auc_score(bldkNNProbR, bldTestY))

"""Accuracy with rgb = around 71 percent
Accuracy with grayscale = around 67 percent
"""

bldPWProbR = PWClassifier(bldTrXG, bldTrY, bldTeXG, len(bldClasses), 1) 
bldPWTestR = bldPWProbR.argmax(axis = 1)

bldPWCM = confMatrix(bldTeY, bldPWTestR, bldClasses)
print("bld PW Accuracy:", accuracy(bldPWCM))
print("bld PW F1 Score:", f1Score(bldPWCM))
print("bld PW AUC Score:", roc_auc_score(bldPWProbR, bldTestY))

"""Accuracy with grayscale = around 53.4 percent

In [ ]:
#Linear Regression with L1,L2,elastic regularization on vegetable dataset, 
#polynomial regression on vegetable dataset, logistic regression on pneumonimnist and bloodmnist
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data = np.load('../input/vegetables/object_localization.npy',encoding = 'latin1',allow_pickle=True)
print(data.shape)
X = data[:,0]
Y = data[:,1:3]
print(X.shape)
print(Y.shape)

# Commented out IPython magic to ensure Python compatibility.
# %matplotlib inline
x=[]
x1 = []
for i in X:
    x.append(i)
    x1.append(i.flatten()/255)
X_input = np.array(x)
X_list = np.array(x1)
print(X_input.shape)
print(X_list.shape)

x=[]
for i in Y:
    x.append(i[1])
Y_coord = np.array(x)
print(Y_coord.shape)

import cv2
from matplotlib import pyplot as plt
def draw_rect(im, cords, color = None):
    im = im.copy()
    
    
    if not color:
        color = [255,0,0]
    pt1 = int(cords[0]),int(cords[1])
    pt2 = int(cords[2]),int(cords[3])
    
    im = cv2.rectangle(im, pt1, pt2, color, int(max(im.shape[:2])/200),)
    plt.imshow(im)
    plt.show()
    return 
draw_rect(X_input[1],Y_coord[1],)

#Code to split the data randomly in 9:1 ratio
test_indices = np.random.choice(X_list.shape[0], size = X_list.shape[0]//10, replace = False)
train_indices = list(filter(lambda x : x not in test_indices, range(X_list.shape[0])))
X_test = X_list[test_indices]
X_train = X_list[train_indices]
Y_test = Y_coord[test_indices]
Y_train = Y_coord[train_indices]

print(X_test.shape, X_train.shape)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

reg = LinearRegression().fit(X_train, Y_train)

Z_test = reg.predict(X_test)
Z_train = reg.predict(X_train)
from sklearn.metrics import mean_squared_error

print(mean_squared_error(Z_test,Y_test))

draw_rect(X_input[test_indices[5]], Z_test[5],)

def linearRegression(X, Y, learning_rate, iterations) :
    X = np.c_[X, np.ones(X.shape[0])]
    n = X.shape[0]
    W = np.zeros((X.shape[1], Y.shape[1]))
    C = []
    
    for _ in range(iterations):
        Y_pred = np.dot(X, W)
        c = np.mean(np.square(Y_pred - Y))/2
        dW = np.dot(X.T, Y_pred - Y)/n
        #print(dW.shape)
        W = W - learning_rate*dW
        C.append(c)
    
    return W, C


def linearRegressionSGD(X, Y, learning_rate, iterations, batch_size) :
    X = np.c_[X, np.ones(X.shape[0])]
    n = X.shape[0]
    W = np.zeros((X.shape[1], Y.shape[1]))
    C = []
    X1 = []
    Y1 = []
    n1 = n - n%batch_size
    for i in range(0,n1,batch_size):
        X1.append(X[i:i+batch_size])
        Y1.append(Y[i:i+batch_size])
    k = len(X1)
    for _ in range(iterations):
        c = 0
        for i in range(k):
            Y_pred = np.dot(X1[i], W)
            c += np.mean(np.square(Y_pred - Y1[i]))/2
            dW = np.dot(X1[i].T, Y_pred - Y1[i])/batch_size
            #print(dW.shape)
            W = W - learning_rate*dW
        print(c/n*batch_size)
        C.append(c/n*batch_size)
        
    return W, C

def linearRegressionL2(X, Y, learning_rate, iterations,alpha,batch_size) :
    X = np.c_[X, np.ones(X.shape[0])]
    n = X.shape[0]
    W = np.zeros((X.shape[1], Y.shape[1]))
    C = []
    
    for _ in range(iterations):
        c=0
        for i in range(0,n,batch_size):
            if i+batch_size < n:
                Y_pred = np.dot(X[i:i+batch_size], W)
                c += np.mean(np.square(Y_pred - Y[i:i+batch_size]))/2 + alpha*np.mean(np.square(W))/2
                dW = np.dot(X[i:i+batch_size].T, Y_pred - Y[i:i+batch_size])/batch_size
            else:
                Y_pred = np.dot(X[i:n], W)
                c += np.mean(np.square(Y_pred - Y[i:n]))/2 + alpha*np.mean(np.square(W))/2
                dW = np.dot(X[i:n].T, Y_pred - Y[i:n])/(n-i)
        W = W - learning_rate*(dW + alpha*W)
        print(c/n*batch_size)
        C.append(c/n*batch_size)
    
    return W, C

def linearRegressionL1(X, Y, learning_rate, iterations,alpha=None) :
    X = np.c_[X, np.ones(X.shape[0])]
    n = X.shape[0]
    W = np.zeros((X.shape[1], Y.shape[1]))
    C = []
    if alpha == None:
        alpha = 0.001
    
    for _ in range(iterations):
        c=0
        for i in range(0,n,batch_size):
            if i+batch_size < n:
                Y_pred = np.dot(X[i:i+batch_size], W)
                c += np.mean(np.square(Y_pred - Y[i:i+batch_size]))/2 + alpha*np.mean(np.abs(W))/2
                dW = np.dot(X[i:i+batch_size].T, Y_pred - Y[i:i+batch_size])/batch_size
            else:
                Y_pred = np.dot(X[i:n], W)
                c += np.mean(np.square(Y_pred - Y[i:n]))/2 + alpha*np.mean(np.abs(W))/2
                dW = np.dot(X[i:n].T, Y_pred - Y[i:n])/(n-i)
        dL1 = W.copy()
        dL1[W>=0]=1
        dL1[W<0]=-1
        
        #print(dW.shape)
        W = W - learning_rate*(dW + alpha*dL1)
        print(c/n*batch_size)
        C.append(c/n*batch_size)
    
    return W, C

def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0

    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

def mean_absolute_error(y1,y2):
    return np.mean(np.abs(y1-y2))

def mean_square_error(y1,y2):
    return np.mean(np.square(y1-y2))

def mean_IOU(y1,y2):
    l = len(y1)
    c=0
    for i in range(l):
        c+= bb_intersection_over_union(y1[i],y2[i])
    return c/l

W, C = linearRegression(X_train, Y_train, 0.0000002, 500)
print(C)
print(X_train.shape)

W, C = linearRegressionSGD(X_train, Y_train, 0.00001, 1000,16)
#print(C)

from sklearn.metrics import mean_squared_error
X = np.c_[X_test, np.ones(X_test.shape[0])]
Z_test_LR = np.dot(X, W)
print(mean_square_error(Z_test_LR,Y_test))

W1,C1= linearRegressionL2(X_train, Y_train, 0.00001, 1000,0.001,16)
W2,C2= linearRegressionL1(X_train, Y_train, 0.00001, 1000,0.001,16)

X_test1 = np.c_[X_test, np.ones(X_test.shape[0])]
W_test1 = np.dot(X_test1, W)
t = np.linspace(0,1,100)
plt.plot(C1,t,C2,t,C,t)
plt.grid()
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend(['L2_Regularization','L1_Regularization','without Regularization'])
plt.show()

draw_rect(X_input[test_indices[150]],Y_test[150],)

import numpy as np

data1 = np.load('../input/pneumoniamnist/pneumoniamnist.npz')
data2 = np.load('../input/bloodmnist/bloodmnist.npz')
print(data1.files)
print(data2.files)

# Reshape images and save in numpy arrays
train1_data = np.reshape(data1['train_images'],(4708,784))
train1_labels = np.reshape(data1['train_labels'],(4708))

print(train1_data.shape)
print(train1_labels.shape)
test1_data = np.reshape(data1['test_images'],(624,784))
test1_labels = np.reshape(data1['test_labels'],(624))
print(test1_data.shape)
print(test1_labels.shape)

def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

def log_likelihood(features, target, weights):
    scores = np.dot(features, weights)
    ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
    return ll

def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))
        
    weights = np.zeros(features.shape[1])
    
    for step in xrange(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)

        # Update weights with log likelihood gradient
        output_error_signal = target - predictions
        
        gradient = np.dot(features.T, output_error_signal)
        weights += learning_rate * gradient

        # Print log-likelihood every so often
        if step % 10000 == 0:
            print log_likelihood(features, target, weights)
        
    return weights

weights = logistic_regression(simulated_separableish_features, simulated_labels,
                     num_steps = 1000, learning_rate = 1e-3, add_intercept=True)



def compute_confusion_matrix(true, pred):

  K = len(np.unique(true)) # Number of classes 
  result = np.zeros((K, K))

  for i in range(len(true)):
    result[true[i]][pred[i]] += 1
  tn = result[0][0] 
  fn = result[0][1] 
  fp = result[1][0] 
  tp = result[1][1]
  acc = (tp+tn)/(tp+tn+fp+fn)
  f1 = tp/(tp+0.5*(fp+fn)) 
  print("Accuracy = ", acc)
  print("F-1 Score = ", f1)
  return result

print(compute_confusion_matrix(test1_labels, predictions))

#print(data2['train_images'].shape)
train2_data = np.reshape(data2['train_images'],(11959,2352))
train2_labels = np.reshape(data2['train_labels'],(11959))

#print(data2['test_images'].shape)
test2_data = np.reshape(data2['test_images'],(3421,2352))
test2_labels = np.reshape(data2['test_labels'],(3421))

def feature_scaling(X):
    mean = np.mean(X_data, axis=0) 
    sd = np.std(X_data, axis=0) 
    X_scaled= (X -  mean) / sd
    
    return X_scaled

def cost_function(theta, X, Y,lambda_reg = 0.0):
   
    m = len(Y)
    A = sigmoid(np.dot(X,theta)) 
    regularization = (lambda_reg/(2 * m)) * np.sum(theta**2)
    cost=(- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A))) + regularization

def gradient(theta, X, Y,lambda_reg = 0.00):
   
    cost,A = cost_function(theta, X, Y)
    m, n = X.shape
    theta = theta.reshape((n, 1))
    gradient = (1 / m) * np.dot(X.T,(A - Y))+ (lambda_reg /m)*theta
    return cost, gradient

def optimize(X, Y, theta, num_iterations , learning_rate,print_cost = False):
    
    costs = []
    
    
    for i in range(num_iterations):
        
        cost, _gradient = gradient(theta, X, Y)
        theta -= learning_rate * _gradient 
        
        costs.append(cost)
        
        # Print the cost every 100 training iterations if required
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        
    return costs,theta.reshape(2352)

def predict(theta, X):
    
    A = sigmoid(np.dot(X,theta.T)) 
    #Predicting the class as the one with highest probability value
    predictions = [classes[np.argmax(A[i, :])] for i in range(X.shape[0])]
        
    return predictions

def accuracy(Y, predictions):
   
    accuracy = sum(predictions == Y)/len(Y)
    
    return accuracy

def model( num_iterations = 2000, learning_rate = 0.0005,print_cost = False, print_accuracy = True):
   
    
    ##Running the implemented code
    classification = []
    logistic_reg_accuracy = np.zeros((10))

    X_data_scaled = feature_scaling(X_data)
    
    ##Iterating over the implemented Logistic Regression for 10 times
    for t in range(10):
        # Splitting the data into traning and testing where training data is 2/3 of dataset randomly with the help of "Shuffle"
        X_train, X_test, Y_train, Y_test = train2_data, test2_data, train2_labels, test2_labels
        classes = np.unique(Y_data)
        theta = np.zeros((8, 2352)) 
        i = 0
        for j in classes:
            #converting the categorical variables to 0 and 1
            Y_temp = np.array(Y_train == j, dtype = int)
            #print(Y_temp)
            Y_temp = Y_temp.reshape((Y_temp.shape[0], 1))
            costs, optimal_theta = optimize(X_train, Y_temp, np.zeros((2352,1)),num_iterations , learning_rate , print_cost)
            theta[i] = optimal_theta
            i += 1
          
        #Predicting for X_test for every iteration
        predictions = predict(theta, X_test)
    
        for j in range(len(predictions)):
            classification.append("%s, %s" %(predictions[j], Y_test[j]))
    
        #Storing the accuracies of each iteration
        logistic_reg_accuracy[t] = accuracy(Y_test, predictions)*100
        if print_accuracy:
            ##Printing the accuracies of each iteration
            print("Accuracy for ", t+1, " iteration: ", logistic_reg_accuracy[t])

    ##printing the average of accuracies across 10 iterations
    print("\nAverage Test Accuracy for Implemented Logistic Regression: ", logistic_reg_accuracy.mean(), '%')

    ##Writing the results to file
    f = open('C:\\Users\\javva\\Downloads\\predictions.csv', 'w')
    for line in classification:
        f.write(line + "\n")
    f.close()
    
    d = {"costs": costs,
         "Y_test": Y_test, 
         "Y_test_predictions" : predictions, 
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations,
         "logistic_reg_accuracy": logistic_reg_accuracy}
    
    return d

d = model(num_iterations = 10000, learning_rate = 0.001)
plt.plot(range(len(d["costs"])),d["costs"],'r')
plt.title("Learning Curve ")
plt.xlabel("Number of Iterations")
plt.ylabel("Cost")
plt.show()

#Confusion matrix for the last iteration of Logistic Regression model
conf_matrix = confusion_matrix(d["Y_test"],d["Y_test_predictions"])
matrix = sns.heatmap(conf_matrix, annot = True, xticklabels = classes, yticklabels = classes)
matrix.set(xlabel = "True Values", ylabel = "False Values")

learning_rates = [0.5,0.1,0.01, 0.001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model( num_iterations = 2000, learning_rate = i, print_accuracy = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations (hundreds)')

legend = plt.legend(loc='upper right', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

class PolynomialRegression(object):

    def __init__(self, x, y):     
        
        self.x = x
        self.y = y      
    
    def standardize(self,data):

        return (data - np.mean(data))/(np.max(data) - np.min(data))
        
    def hypothesis(self, theta, x):
         
        h = theta[0]
        for i in np.arange(1, len(theta)):
            h += theta[i]*x ** i        
        return h        
        
    def computeCost(self, x, y, theta):
   
        m = len(y)  
        h = self.hypothesis(theta, x)
        errors = h-y
        
        return (1/(2*m))*np.sum(errors**2) 
        
    def fit(self, method = 'normal_equation', order = 1, tol = 10**-3, numIters = 20, learningRate = 0.01):


        if method == 'normal_equation': 
            d = {}
            d['x' + str(0)] = np.ones([1,len(x_pts)])[0]    
            for i in np.arange(1, order+1):                
                d['x' + str(i)] = self.x ** (i)        
                
            d = OrderedDict(sorted(d.items(), key=lambda t: t[0]))
            X = np.column_stack(d.values())  

            theta = np.matmul(np.matmul(linalg.pinv(np.matmul(np.transpose(X),X)), np.transpose(X)), self.y)

        elif method == 'gradient_descent':
                
            d = {}
            d['x' + str(0)] = np.ones([1,len(x_pts)])[0]    
            for i in np.arange(1, order+1):                
                d['x' + str(i)] = self.standardize(self.x ** (i))      
                
            d = OrderedDict(sorted(d.items(), key=lambda t: t[0]))
            X = np.column_stack(d.values())  
                
            m = len(self.x)
            theta = np.zeros(order + 1)           
            costs = []
            for i in range(numIters):
             
                h = self.hypothesis(theta, self.x)       
                errors = h-self.y
                theta += -learningRate * (1/m)*np.dot(errors, X)
                cost = self.computeCost(self.x, self.y, theta)
                costs.append(cost)         
                #tolerance check
                if cost < tol:
                    break
                
            self.costs = costs
            self.numIters = numIters
            
        self.method = method    
        self.theta = theta        

        return self
        
    def plot_predictedPolyLine(self):
               
        plt.figure()
        plt.scatter(self.x, self.y, s = 30, c = 'b') 
        line = self.theta[0] #y-intercept 
        label_holder = []
        label_holder.append('%.*f' % (2, self.theta[0]))
        for i in np.arange(1, len(self.theta)):            
            line += self.theta[i] * self.x ** i 
            label_holder.append(' + ' +'%.*f' % (2, self.theta[i]) + r'$x^' + str(i) + '$') 

        plt.plot(self.x, line, label = ''.join(label_holder))        
        plt.title('Polynomial Fit: Order ' + str(len(self.theta)-1))
        plt.xlabel('x')
        plt.ylabel('y') 
        plt.legend(loc = 'best')      

    def plotCost(self):
           
        if self.method == 'gradient_descent':
            plt.figure()
            plt.plot(np.arange(1, self.numIters+1), self.costs, label = r'$J(\theta)$')
            plt.xlabel('Iterations')
            plt.ylabel(r'$J(\theta)$')
            plt.title('Cost vs Iterations of Gradient Descent')
            plt.legend(loc = 'best')
        else:
            print('plotCost method can only be called when using gradient descent method')
  
            

In [ ]:
#GMM for tiny imagenet
"""Q5 GMM as a Generative Model"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from skimage.io import imread
from skimage.transform import downscale_local_mean, rescale
from tqdm import tqdm
from numba import jit
from skimage.filters import unsharp_mask
import os

def process_data(path):
    i = 0
    dataset = np.zeros((500, 8*2, 8*2, 3))
    dataset_ = np.zeros((500, 4*8*8*3))
    for file_ in os.listdir(path):
        if imread(path+'/'+file_).shape == (64, 64, 3):
            dataset[i, :, :, :] = downscale_local_mean(imread(path+'/'+file_)/255.0, (4, 4, 1))
        i += 1

    dataset_ = np.reshape(dataset, (500, 4*8*8*3))
    return dataset_


def density_estimate(x, N):
    gm = GaussianMixture(n_components=N, random_state=0).fit(x)
    return gm


def calc_pdf(x, prior, lambdas, means, covariances, N):
    s = 0.0
    for i in range(N):
        s += lambdas[i]*multivariate_normal.pdf(x, mean=means[i, :], cov=covariances[i, :, :])
    posterior = prior * s
    return posterior

def main():
    path = '/content/drive/MyDrive/PRNN/jellyfish/images'
    dataset = process_data(path)
    f = density_estimate(dataset, 5)
    c = f.sample(16)

    rows = cols = 4
    fig2, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(5,5))
    for idx in range(rows*cols):
        d = c[0][idx]
        d = d-np.min(d)
        d = d/np.max(d)
        image = np.reshape(d, (8*2, 8*2, 3))
        #image = rescale(np.reshape(d, (8*2, 8*2, 3)), (4, 4, 1))
        row = idx // cols
        col = idx % cols
        axes[row, col].axis("off")
        axes[row, col].imshow(image, aspect="auto")
    plt.subplots_adjust(wspace=.01, hspace=.01)
    plt.show()


if __name__ == "__main__":
    main()

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.nn.functional import adaptive_avg_pool2d
import torchvision.models as models
import torch.nn.functional as F
from scipy import linalg

class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp
    
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
fid_model = InceptionV3([block_idx])
fid_model = fid_model.cuda()

def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
  
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

def calculate_fretchet(images_real,images_fake,model):
     mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
     mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
   
     fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
     return fid_value

#Calculate FID score of generated sample
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model = model.cuda()
avg_fid_score = 0.0
img_size=(h_image,w_image)
h_image = 32//2
w_image = 32//2

for i in range(10):
    d = c[0][i]
    d = d-np.min(d)
    d = d/np.max(d)
    image = np.reshape(d, (8*2, 8*2, 3))
    c = resize(np.asarray(X_test[i, :, :, 0]), (64, 64), anti_aliasing=True)
    d = resize(np.asarray(output), (64, 64), anti_aliasing=True)
    a = torch.from_numpy(c)
    b = torch.from_numpy(d)
    a = torch.from_numpy(np.expand_dims(a, 0)).float()
    b = torch.from_numpy(np.expand_dims(b, 0)).float()
    avg_fid_score += calculate_fretchet(a, b, model)
print(avg_fid_score/10.0)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
from skimage.transform import downscale_local_mean
from tqdm import tqdm
from numba import jit
from sklearn import metrics

def process_data(path):

    dataset = np.load(path)
    train_x, train_y = dataset['train_images']/255.0, dataset['train_labels']
    val_x, val_y = dataset['val_images']/255.0, dataset['val_labels']
    test_x, test_y = dataset['test_images']/255.0, dataset['test_labels']

    scale = 4

    train_x_ = np.zeros((train_x.shape[0], train_x.shape[1]**2//scale**2))
    val_x_ = np.zeros((val_x.shape[0], val_x.shape[1]**2//scale**2))
    test_x_ = np.zeros((test_x.shape[0], test_x.shape[1]**2//scale**2))

    for i in range(train_x.shape[0]):
        temp = downscale_local_mean(train_x[i, :, :], (scale, scale))
        train_x_[i, :] = temp.reshape(train_x.shape[1]**2//scale**2)
        #train_x_[i, :] = train_x_[i, :]/np.max(train_x_[i, :])

    for i in range(val_x.shape[0]):
        temp = downscale_local_mean(val_x[i, :, :], (scale, scale))
        val_x_[i, :] = temp.reshape(val_x.shape[1]**2//scale**2)
        #val_x_[i, :] = val_x_[i, :]/np.max(val_x_[i, :])

    for i in range(test_x.shape[0]):
        temp = downscale_local_mean(test_x[i, :, :], (scale, scale))
        test_x_[i, :] = temp.reshape(test_x.shape[1]**2//scale**2)
        #test_x_[i, :] = test_x_[i, :]/np.max(test_x_[i, :])

    return train_x_, train_y, test_x_, test_y, val_x_, val_y


def class_conditionals(x, y):
    x_0 = np.zeros((np.sum(y==0), x.shape[1]))
    y_0 = np.zeros((np.sum(y==0)))
    x_1 = np.zeros((np.sum(y==1), x.shape[1]))
    y_1 = np.ones((np.sum(y==1)))

    p0 = (np.sum(y==0))/y.shape[0]
    p1 = (np.sum(y==1))/y.shape[0]

    i_0 = 0
    i_1 = 0
    for i in range(len(y)):
        if(y[i] == 0):
            x_0[i_0, :] = x[i, :]
            i_0 += 1
        else:
            x_1[i_1, :] = x[i, :]
            i_1 += 1
    return x_0, y_0, x_1, y_1, p0, p1



def phi(x, mu, sigma, k):
    if np.isfinite(np.linalg.cond(sigma)):
        sigma = sigma + np.eye(sigma.shape[0])
    
    out = multivariate_normal(mu, sigma).pdf(x)
    return out


def em(data, K, iters):
    # K = Number of mixture densities, N = Number of data samples
    N = data.shape[0]
    
    # Initialize the parameters - mixing coefficients, means and covariances
    lambdas = (1/K)*np.ones(K)
    means = np.zeros((data.shape[1], K))
    sigmas = np.zeros((K, data.shape[1], data.shape[1]))
    for i in range(K):
        sigmas[i, :, :] = np.eye(data.shape[1])
    gamma = np.ones((N, K))
    
    for k in tqdm(range(iters)):
        # E Step
        for i in range(N):
            for j in range(K):                    
                gamma[i, j] = lambdas[j]*phi(data[i, :], means[:, j], sigmas[j, :, :], [i, j, k])
                sums = 0.0
                for l in range(K):
                    sums += lambdas[l]*phi(data[i, :], means[:, l], sigmas[l, :, :], [i, j, k])

                if not(sums == 0):
                    gamma[i, j] = gamma[i, j] / sums
                else:
                    gamma[i, j] = gamma[i, j] / (0.00001 + sums)

        # M Step
        # Sum over rows of gamma for all data samples
        denominator = np.sum(gamma, axis=0)

        for j in range(K):
            mean_sums = np.zeros(data.shape[1])
            cov_sums = np.zeros((data.shape[1], data.shape[1]))
            for i in range(N):
                mean_sums += gamma[i, j]*data[i, :]
                cov_sums += gamma[i, j]*np.outer((data[i, :]-means[:, j]),
                                                 (data[i, :]-means[:, j]).transpose())

            means[:, j] = mean_sums / \
                denominator[j] if not(denominator[j] == 0) else mean_sums / \
                (0.00001 + denominator[j])
            sigmas[j, :, :] = cov_sums / \
                denominator[j] if not(denominator[j] == 0) else cov_sums / \
                (0.00001 + denominator[j])
        lambdas = (1/N)*denominator

    params = [lambdas, means, sigmas]
    return params


def calc_pdf(x, prior, lambdas, means, covariances, N):
    s = 0.0
    for i in range(N):
        s += lambdas[i]*multivariate_normal.pdf(x, mean=means[i, :], cov=covariances[i, :, :])
    posterior = prior * s
    return posterior

def bayes_with_gmms(train_x, train_y, test_x, test_y, N):
    print("\n[INFO] Fitting the data to a GMM using EM \n")
    # Fit the GMMs with training data
    x_0, y_0, x_1, y_1, p0, p1 = class_conditionals(train_x, train_y)
    em_iters = 5
    
    ####
    f0_lambdas, f0_means, f0_covariances = em(x_0, N, em_iters)
    lambdas0, means0, covariances0 = f0_lambdas, f0_means.transpose(), f0_covariances
    
    ###
    f1_lambdas, f1_means, f1_covariances = em(x_1, N, em_iters)
    lambdas1, means1, covariances1 = f1_lambdas, f1_means.transpose(), f1_covariances
    
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    print("[INFO] Calculating accuracy and F1 score on test data for GMM with ", N, " component(s)\n")

    test_preds = np.zeros_like(test_y)
    for i in range(test_y.shape[0]):
        q1 = calc_pdf(test_x[i, :], p1, lambdas1, means1, covariances1, N)
        q0 = calc_pdf(test_x[i, :], p0, lambdas0, means0, covariances0, N)
        if(q1 >= q0):
            pred_class = 1
            test_preds[i] = 1
            if(test_y[i] == pred_class):
                tp += 1
            else:
                fp += 1
        else:
            pred_class = 0
            test_preds[i] = 0
            if(test_y[i] == pred_class):
                tn += 1
            else:
                fn += 1

    acc = (tp+tn)/(tp+tn+fp+fn)
    f1 = tp/(tp+0.5*(fp+fn))
    print("\nAccuracy = ", acc)
    print("F-1 Score = ", f1, "\n\n")    
    return test_preds

def main():
    path = '/content/drive/MyDrive/PRNN/pneumoniamnist.npz'
    train_x, train_y, test_x, test_y, val_x, val_y = process_data(path)
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 1)
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 2)
    test_preds = bayes_with_gmms(train_x, train_y, val_x, val_y, 3)
    print(sklearn.metrics.f1_score(test_y, test_preds))
    print(sklearn.metrics.roc_auc_score(test_y, test_preds))
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 4)
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 5)
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 6)
    #bayes_with_gmms(train_x, train_y, val_x, val_y, 7)


if __name__ == "__main__":
    main()

"""MAP with Gaussian Class conditionals and Gaussian prior"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from skimage.transform import downscale_local_mean

def process_data(path):

    dataset = np.load(path)
    train_x, train_y = dataset['train_images']/255.0, dataset['train_labels']
    val_x, val_y = dataset['val_images']/255.0, dataset['val_labels']
    test_x, test_y = dataset['test_images']/255.0, dataset['test_labels']

    scale = 4

    train_x_ = np.zeros((train_x.shape[0], train_x.shape[1]**2//scale**2))
    val_x_ = np.zeros((val_x.shape[0], val_x.shape[1]**2//scale**2))
    test_x_ = np.zeros((test_x.shape[0], test_x.shape[1]**2//scale**2))

    for i in range(train_x.shape[0]):
        temp = downscale_local_mean(train_x[i, :, :], (scale, scale))
        train_x_[i, :] = temp.reshape(train_x.shape[1]**2//scale**2)
        #train_x_[i, :] = train_x_[i, :]/np.max(train_x_[i, :])

    for i in range(val_x.shape[0]):
        temp = downscale_local_mean(val_x[i, :, :], (scale, scale))
        val_x_[i, :] = temp.reshape(val_x.shape[1]**2//scale**2)
        #val_x_[i, :] = val_x_[i, :]/np.max(val_x_[i, :])

    for i in range(test_x.shape[0]):
        temp = downscale_local_mean(test_x[i, :, :], (scale, scale))
        test_x_[i, :] = temp.reshape(test_x.shape[1]**2//scale**2)
        #test_x_[i, :] = test_x_[i, :]/np.max(test_x_[i, :])

    return train_x_, train_y, test_x_, test_y, val_x_, val_y


def bayes_classifier(train_x, train_y, prior_mu0, prior_mu1):
    
    sum0 = np.zeros((train_x[1].shape[0]))
    sum1 = np.zeros((train_x[1].shape[0]))
    mean0 = np.zeros((train_x[1].shape[0]))
    mean1 = np.zeros((train_x[1].shape[0]))   
    sig0 = np.zeros((train_x[1].shape[0], train_x[1].shape[0]), dtype=np.double)
    sig1 = np.zeros((train_x[1].shape[0], train_x[1].shape[0]), dtype=np.double)
    sig0_N = np.zeros((train_x[1].shape[0], train_x[1].shape[0]), dtype=np.double)
    sig1_N = np.zeros((train_x[1].shape[0], train_x[1].shape[0]), dtype=np.double)
    prior_sig = np.eye(train_x[1].shape[0])
    n0 = 0
    n1 = 0
    n = train_x.shape[0]

    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            sum0 += train_x[i]
            n0 += 1
        else:
            sum1 += train_x[i]
            n1 += 1


    print("[INFO] Estimating class conditional densities")

    for i in range(train_y.shape[0]):
        if(train_y[i] == 0):
            sig0 += np.outer(train_x[i]-sum0/n0, (train_x[i]-sum0/n0).T)
        else:
            sig1 += np.outer(train_x[i]-sum1/n1, (train_x[i]-sum1/n1).T)

    sig0 *= 1.0/n0
    sig1 *= 1.0/n1

    sig0_N = np.linalg.pinv(prior_sig + n0*np.linalg.pinv(sig0))
    sig1_N = np.linalg.pinv(prior_sig + n1*np.linalg.pinv(sig1))
    
    mean0 = np.matmul(sig0_N, np.matmul(prior_sig, prior_mu0) + np.matmul(np.linalg.pinv(sig0), sum0))
    mean1 = np.matmul(sig1_N, np.matmul(prior_sig, prior_mu1) + np.matmul(np.linalg.pinv(sig1), sum1))

    p0 = n0/n
    p1 = n1/n

    (sign0, logdet0) = np.linalg.slogdet(sig0_N)
    (sign1, logdet1) = np.linalg.slogdet(sig1_N)

    return mean0, mean1, sig0_N, sig1_N, logdet0, sign0, logdet1, sign1, p0, p1


def testing_bayes(test_x, test_y, mu0, mu1, sig0, sig1, logdet0, sign0, logdet1, sign1, p0, p1):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    print("[INFO] Calculating accuracy and F1 score on test data")

    for i in range(test_y.shape[0]):
        X = test_x[i, :]
        log_q1 = np.log(p1) - 0.5*logdet1 - 0.5*np.matmul((X-mu1).transpose(),
                                                          np.matmul(np.linalg.pinv(sig1), X-mu1))
        log_q0 = np.log(p0) - 0.5*logdet0 - 0.5*np.matmul((X-mu0).transpose(),
                                                          np.matmul(np.linalg.pinv(sig0), X-mu0))

        if(log_q1 >= log_q0):
            pred_class = 1
            if(test_y[i] == pred_class):
                tp += 1
            else:
                fp += 1
        else:
            pred_class = 0
            if(test_y[i] == pred_class):
                tn += 1
            else:
                fn += 1

    acc = (tp+tn)/(tp+tn+fp+fn)
    f1 = tp/(tp+0.5*(fp+fn))
    print("Accuracy = ", acc)
    print("F-1 Score = ", f1)
    return


def main():
    path =  '/content/drive/MyDrive/PRNN/pneumoniamnist.npz'
    train_x, train_y, test_x, test_y, val_x, val_y = process_data(path)
    mu0, mu1, sig0, sig1, logdet0, sign0, logdet1, sign1, p0, p1 = bayes_classifier(
        train_x, train_y, np.random.normal(size=(49))*10000000000, np.random.normal(size=(49))*-5000000000)
    testing_bayes(test_x, test_y, mu0, mu1, sig0, sig1, logdet0, sign0, logdet1, sign1, p0, p1)


if __name__ == "__main__":
    main()

